## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-27-04-16-31 +0000,bbc,At least 10 FBI agents who worked on Trump inv...,https://www.bbc.com/news/articles/c4gq7vdld80o...
1,2026-02-27-04-15-17 +0000,nyt,"After F.B.I. Raid, Los Angeles School Board Di...",https://www.nytimes.com/2026/02/26/us/los-ange...
2,2026-02-27-04-02-46 +0000,bbc,"Netflix drops bid for Warner Bros, clearing wa...",https://www.bbc.com/news/articles/c5y6p5ypgmzo...
3,2026-02-27-03-54-27 +0000,bbc,Anthropic boss rejects Pentagon demand to drop...,https://www.bbc.com/news/articles/cvg3vlzzkqeo...
4,2026-02-27-03-51-00 +0000,wsj,Anthropic is standing firm in its dispute with...,https://www.wsj.com/politics/national-security...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2522/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,51
167,epstein,20
30,ai,18
76,iran,17
61,out,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
217,2026-02-26-19-24-19 +0000,nypost,DOJ probes whether any Epstein files about unp...,https://nypost.com/2026/02/26/us-news/doj-prob...,105
85,2026-02-27-00-03-07 +0000,nypost,House Dems to force vote to block Trump from w...,https://nypost.com/2026/02/26/us-news/house-de...,103
336,2026-02-26-12-32-24 +0000,nypost,"Latest US, Iran talks conclude in Geneva as Tr...",https://nypost.com/2026/02/26/us-news/new-us-i...,103
177,2026-02-26-21-28-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...,102
78,2026-02-27-00-25-42 +0000,nypost,Trump moves more military might to the Middle ...,https://nypost.com/2026/02/26/us-news/trump-mo...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
217,105,2026-02-26-19-24-19 +0000,nypost,DOJ probes whether any Epstein files about unp...,https://nypost.com/2026/02/26/us-news/doj-prob...
150,95,2026-02-26-22-06-57 +0000,nypost,Paramount Skydance comes out on top of bidding...,https://nypost.com/2026/02/26/business/warner-...
35,78,2026-02-27-02-13-00 +0000,wsj,Hillary Clinton said that she had no knowledge...,https://www.wsj.com/politics/republicans-press...
354,62,2026-02-26-11-22-15 +0000,cbc,Iran nuclear talks conclude with no deal annou...,https://www.cbc.ca/news/world/geneva-us-iran-n...
317,61,2026-02-26-14-25-08 +0000,bbc,A New York City snowball fight gone wrong leav...,https://www.bbc.com/news/articles/cwyxlgkj0k3o...
259,51,2026-02-26-17-48-00 +0000,wsj,Mortgage rates fell below 6% this week for the...,https://www.wsj.com/economy/housing/mortgage-r...
120,47,2026-02-26-22-49-32 +0000,latimes,Downtown LA Law Group moves to block state bar...,https://www.latimes.com/california/story/2026-...
177,47,2026-02-26-21-28-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...
359,43,2026-02-26-11-00-00 +0000,latimes,Hiltzik: AI isn't ready to be your doctor yet ...,https://www.latimes.com/business/story/2026-02...
101,39,2026-02-26-23-22-46 +0000,nypost,Kim Jong Un’s sister makes first public appear...,https://nypost.com/2026/02/26/world-news/kim-j...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
